Make that we are running the correct version of TensorFlow first

In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [2]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Train basic model on the generated emnist-lines dataset

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
# To be able to write code as if this notebook was one level up in files tree structure.
sys.path.append('..')

from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.ops import ctc_ops
from tensorflow.python.ops import math_ops as tf_math_ops

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Input, Lambda, Reshape, TimeDistributed, LSTM
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence

import pandas as pd
from PIL import Image

In [4]:
from recognizer.datasets import Dataset
from recognizer.datasets import EmnistDataset
from recognizer.networks import lenet
from recognizer.networks import simple
from recognizer.networks import NetworkInput
from recognizer.models import CTCModel

Hyperparameters

In [5]:
batch_size = 256
epochs = 4  # 32

In [6]:
model_save_path = Path("../recognizer/weights/lines_time_distributed_model.h5")

## Load characters mapping

In [7]:
import json

with open(Dataset.raw_data_path()/"emnist"/"emnist_essentials.json") as json_file:  
    mapping = json.load(json_file)["mapping"]
    mapping = {m[0]: m[1] for m in mapping}
    mapping[62] = ' '
    mapping[63] = '_'
    
mapping_reversed = {v: k for k, v in mapping.items()}

## Build the new model

In [8]:
char_image_height = 28
char_image_width = 28
max_length = 34
num_classes = 64
image_height = char_image_height
image_width = char_image_width * max_length

input_shape = (image_height, image_width)
output_shape = (max_length, num_classes)

window_width: float = 16
window_stride: float = 8
    
num_windows = ((image_width - window_width) // window_stride) + 1
if num_windows < max_length:
    raise ValueError(f'Window width/stride need to generate >= {max_length} windows (currently {num_windows})')

In [9]:
num_windows

118

In [10]:
def slide_window(image, window_width, window_stride):
    """
    Takes (image_height, image_width, 1) input,
    Returns (num_windows, image_height, window_width, 1) output, where
    num_windows is floor((image_width - window_width) / window_stride) + 1
    """
    patches = tf.image.extract_image_patches(image, 
                                             sizes=[1, 1, window_width, 1], 
                                             strides=[1, 1, window_stride, 1], 
                                             rates=[1, 1, 1, 1], 
                                             padding='VALID')
    patches = tf.transpose(patches, (0, 2, 1, 3))
    patches = tf.expand_dims(patches, -1)
    
    return patches

In [11]:
window_width = 12
window_stride = 5

image_input = Input(shape=input_shape, name='image')
image_reshaped = Reshape((image_height, image_width, 1))(image_input)

image_patches = Lambda(
    slide_window,
    arguments={'window_width': window_width, 'window_stride': window_stride}
)(image_reshaped)

In [12]:
image_patches

<tf.Tensor 'lambda/ExpandDims:0' shape=(None, 189, 28, 12, 1) dtype=float32>

### The model

In [13]:
convnet = lenet(NetworkInput(input_shape=(image_height, window_width, 1), number_of_classes=num_classes))
convnet.layers

In [14]:
# Get rid of the last two layers (dropout and softmax)
convnet = KerasModel(inputs=convnet.inputs, outputs=convnet.layers[-2].output)
time_distributed_outputs = TimeDistributed(convnet)(image_patches)

lstm_outputs = LSTM(128, return_sequences=True)(time_distributed_outputs)

softmax_output = Dense(num_classes, activation='softmax', name='softmax_output')(lstm_outputs)

model = CTCModel([image_input], [softmax_output])

## Look at the data

In [15]:
df = pd.read_csv(Dataset.processed_data_path()/"emnist_lines"/"data.csv")

output_length = len(df)

In [16]:
df.head()

,image,sentence
0,0.png,Whats wrong at state _____________
1,1.png,The transducer itself moves the __
2,2.png,There is an ancient and venerable
3,3.png,They destroyed a trading house ___
4,4.png,Alec waited a moment on guard ___


In [17]:
image = Image.open(Dataset.processed_data_path()/"emnist_lines"/"0.png")
image

In [18]:
np.array(image).shape

(28, 952)

## Train it

In [19]:
def format_batch_ctc(batch_x, batch_y):
    """
    Because CTC loss needs to be computed inside of the network, we include information about outputs in the inputs.
    """
    batch_size = batch_y.shape[0]
    y_true = np.argmax(batch_y, axis=-1)
    
    label_lengths = []
    for ind in range(batch_size):
        # Find all of the indices in the label that are blank
        empty_at = np.where(batch_y[ind, :, -1] == 1)[0]
        # Length of the label is the pos of the first blank, or the max length
        if empty_at.shape[0] > 0:
            label_lengths.append(empty_at[0])
        else:
            label_lengths.append(batch_y.shape[1])

    batch_inputs = {
        'image': batch_x,
        'labels': y_true,
        'input_length': np.ones((batch_size, 1)),  # dummy, will be set to num_windows in network
        'label_length': np.array(label_lengths)
    }
    
    return batch_inputs, batch_y
#     batch_outputs = {
#         'ctc_loss': np.zeros(batch_size),  # dummy
#         'ctc_decoded': y_true
#     }
    
#     return batch_inputs, batch_outputs

In [20]:
class LinesDataSequence(Sequence):
    
    def __init__(self, batch_size=32, augment_fn=None, format_fn=None):
        self.df = pd.read_csv(Dataset.processed_data_path()/"emnist_lines"/"data.csv")
        # Shuffle data and reset their index.
        self.df = self.df.sample(frac=1).reset_index(drop=True)
        
        self.batch_size = batch_size
        self.augment_fn = augment_fn
        self.format_fn = format_fn

    def __len__(self):
        return len(df) // batch_size

    def __getitem__(self, idx):
        """Return a single batch."""
        begin = idx * self.batch_size
        end = (idx + 1) * self.batch_size

        batch_x = []
        batch_y = []
        for index in range(begin, end):
            row = self.df.iloc[index]
            
            image = Image.open(Dataset.processed_data_path()/"emnist_lines"/row['image'])
            x = np.array(image).astype(np.float32).reshape(image_height, image_width)
            batch_x.append(x)

            y = [mapping_reversed[char] for char in row['sentence']]
            y = to_categorical(y, num_classes).astype(np.int)
            batch_y.append(y)
            
        batch_x, batch_y = np.array(batch_x), np.array(batch_y)

        return format_batch_ctc(batch_x, batch_y)

In [21]:
callbacks = [
    keras.callbacks.TensorBoard(log_dir='../logs/lines_time_distributed')
]

# model.compile(optimizer=RMSprop(), metrics=['loss', 'accuracy'])
model.compile(Adam(lr=0.0001))

W0802 12:47:55.915802 4359370176 deprecation.py:323] From /Users/mostafagazar/anaconda3/envs/line-reader/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:5151: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0802 12:47:56.033132 4359370176 deprecation.py:323] From /Users/mostafagazar/anaconda3/envs/line-reader/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:5130: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0802 12:47:56.051226 4359370176 deprecation.py:323] From /Users/mostafagazar/anaconda3/envs/line-reader/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:5207: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` an

In [22]:
# model.model_init.summary()
model.model_train.summary()
# model.model_eval.summary()
# model.model_pred.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 28, 952)]    0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 28, 952, 1)   0           image[0][0]                      
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 189, 28, 12,  0           reshape[0][0]                    
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 189, 128)     412160      lambda[0][0]                     
____________________________________________________________________________________________

Check progress in TensorBoard by running `tensorboard --logdir=logs/lines_time_distributed` in terminal

In [23]:
model.fit_generator(LinesDataSequence(batch_size),
                    steps_per_epoch=len(df) // batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    workers=2)

Epoch 1/4


InvalidArgumentError: Not enough time for target transition sequence (required: 25, available: 1)0You can turn this error into a warning by using the flag ignore_longer_outputs_than_inputs
	 [[{{node CTCloss/CTCLoss}}]] [Op:__inference_keras_scratch_graph_3271]

# Save the model

In [ ]:
model.save(model_save_path)

# keras.models.save_model(
#     model,
#     model_save_path,
#     overwrite=True,
#     include_optimizer=True
# )

## Restore the model

In [ ]:
# model = keras.models.load_model(model_save_path)

## Evaluate